In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# 2nd eLU, tanh, 4, 50, 50, 50, 1
## Model
## Train NN & Predict

Import Data (normalized) & Separate to Training and Test

In [3]:
#data = np.loadtxt("ACL nomalization data.txt")
#data2 = np.loadtxt("acl data.txt")
data = np.loadtxt("ACL FTIR data_mean.txt")

X_NormalizedData = data[:,0:4]
Y_NormalizedData = data[:,4]

X_TrainingData = data[0:20,0:4]
Y_TrainingData = data[0:20,4]

X_TestData = data[20:,0:4]
Y_TestData = data[20:,4]

Sequence = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

#XY_TrainingData = data[0:21,:]
#X_TrainingData = XY_TrainingData[0:21,0:4]
#Y_TrainingData = XY_TrainingData[0:21,4]
#
#XY_TestData = data[21:,:]
#X_TestData = XY_TestData[:,0:4]
#Y_TestData = XY_TestData[:,4]

print(X_TrainingData)
print("------------------------")
print(X_TestData)
print("------------------------")
print(Y_TrainingData)
print("------------------------")
print(Y_TestData)

[[ 0.5  0.5  1.   1. ]
 [ 0.5  0.   0.5  1. ]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.5  1.   0.5]
 [ 1.   0.5  0.5  1. ]
 [ 0.5  0.5  0.   1. ]
 [ 1.   0.   0.5  0.5]
 [ 0.5  0.5  0.   0. ]
 [ 0.5  0.   0.   0.5]
 [ 0.5  0.5  1.   0. ]
 [ 0.   0.5  0.5  0. ]
 [ 1.   1.   0.5  0.5]
 [ 0.5  0.   1.   0.5]
 [ 0.   1.   0.5  0.5]
 [ 1.   0.5  0.5  0. ]
 [ 0.   0.5  0.   0.5]
 [ 0.5  1.   1.   0.5]
 [ 0.5  1.   0.5  0. ]
 [ 1.   0.5  1.   0.5]
 [ 0.5  1.   0.   0.5]]
------------------------
[[ 0.5  0.   0.5  0. ]
 [ 1.   0.5  0.   0.5]
 [ 0.   0.5  0.5  1. ]
 [ 0.5  0.5  0.5  0.5]
 [ 0.5  1.   0.5  1. ]]
------------------------
[-0.21844449 -0.47152439 -0.45735005 -0.1123416  -0.11245536 -0.18036006
 -0.19692817  0.00145492 -0.14520267  0.01178692  0.04523337  0.52847561
 -0.13421128  0.19149351 -0.00787756  0.09143864 -0.16016158  0.30505231
  0.04222738  0.33060721]
------------------------
[-0.08104142  0.20294599  0.18485121  0.07951933  0.2628122 ]


# 1st eLU, tanh, 4, 18, 18, 9, 9, 1
   R2 = 0.650794040271

## Model

In [3]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=18
n_2ndHiddenUnit=18
n_3rdHiddenUnit=9
n_4thHiddenUnit=9
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [4]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0591896
11 1000 0.0932284
5 2000 0.0952678
8 3000 0.0939003
14 4000 0.095178
10 5000 0.0968684
14 6000 0.0933779
12 7000 0.0960672
0 8000 0.0976867
11 9000 0.0920585
9 10000 0.0954067
16 11000 0.103425
17 12000 0.100246
9 13000 0.108791
18 14000 0.0992377
6 15000 0.109174
4 16000 0.102449
4 17000 0.108634
5 18000 0.104711
14 19000 0.106914
4 20000 0.100476
14 21000 0.111104
1 22000 0.114193
3 23000 0.11476
8 24000 0.116818
2 25000 0.105358
10 26000 0.107656
3 27000 0.118919
12 28000 0.110507
0 29000 0.111077
11 30000 0.115278
13 31000 0.113085
15 32000 0.110596
16 33000 0.114103
12 34000 0.10827
13 35000 0.105011
5 36000 0.107045
10 37000 0.106673
13 38000 0.120903
2 39000 0.11023
12 40000 0.109934
5 41000 0.106067
8 42000 0.110262
13 43000 0.118721
2 44000 0.110114
3 45000 0.113067
4 46000 0.114026
18 47000 0.106068
4 48000 0.113776
6 49000 0.11248
9 50000 0.115551
12 51000 0.112386
17 52000 0.114736
18 53000 0.112121
12 54000 0.115675
19 55000 0.117282
19 56000 0.117015
0 57000

# 2nd eLU, tanh, 4, 50, 50, 50, 50, 1
R2 = 0.739438404489

## Model

In [5]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=50
n_3rdHiddenUnit=50
n_4thHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [6]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

7 0 0.0595104
19 1000 0.0794318
1 2000 0.0947253
3 3000 0.0882092
8 4000 0.0973845
3 5000 0.0912867
13 6000 0.109581
14 7000 0.0984348
16 8000 0.106975
6 9000 0.104623
14 10000 0.112115
6 11000 0.117953
1 12000 0.112353
1 13000 0.110814
8 14000 0.114513
13 15000 0.122464
17 16000 0.11721
19 17000 0.111499
14 18000 0.111959
17 19000 0.120489
19 20000 0.111344
0 21000 0.113771
15 22000 0.117568
6 23000 0.110788
11 24000 0.114281
10 25000 0.107376
4 26000 0.116347
3 27000 0.120781
18 28000 0.112704
9 29000 0.112731
11 30000 0.120121
12 31000 0.116988
6 32000 0.115963
0 33000 0.116304
0 34000 0.114918
19 35000 0.116479
4 36000 0.122564
19 37000 0.114276
3 38000 0.117473
0 39000 0.115592
2 40000 0.119671
10 41000 0.115765
16 42000 0.114277
7 43000 0.115904
13 44000 0.113214
1 45000 0.115032
9 46000 0.112982
13 47000 0.114722
17 48000 0.11347
3 49000 0.116227
9 50000 0.114015
18 51000 0.117751
12 52000 0.112584
17 53000 0.115038
14 54000 0.11479
13 55000 0.108616
14 56000 0.113845
19 57000 0

# 3rd eLU, tanh, 4, 10, 10, 10, 1
R2 = 0.175927828405

## Model

In [7]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=10
n_3rdHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

9 0 0.0585732
18 1000 0.0956587
13 2000 0.0985326
9 3000 0.0949933
10 4000 0.095396
16 5000 0.0974526
10 6000 0.0956013
1 7000 0.0963442
14 8000 0.100505
18 9000 0.0994718
15 10000 0.096631
3 11000 0.0963814
5 12000 0.099483
14 13000 0.0987677
15 14000 0.0976411
3 15000 0.100531
13 16000 0.101459
14 17000 0.0999619
3 18000 0.103314
0 19000 0.103218
13 20000 0.100759
3 21000 0.102449
1 22000 0.103823
4 23000 0.102486
5 24000 0.105033
11 25000 0.104035
8 26000 0.10446
11 27000 0.104431
4 28000 0.104472
0 29000 0.10785
10 30000 0.106745
11 31000 0.104375
0 32000 0.102216
16 33000 0.108761
1 34000 0.107189
5 35000 0.101689
8 36000 0.108072
7 37000 0.106626
0 38000 0.10947
4 39000 0.110847
18 40000 0.110233
10 41000 0.10897
18 42000 0.109099
19 43000 0.115332
2 44000 0.103648
18 45000 0.106616
11 46000 0.110686
17 47000 0.103853
17 48000 0.111057
12 49000 0.107612
10 50000 0.115527
4 51000 0.113622
17 52000 0.114835
4 53000 0.112899
10 54000 0.117094
17 55000 0.106935
19 56000 0.116198
14 5

# 4th eLU, tanh, 4, 50, 30, 10, 1
R2 = 0.719074577467

## Model

In [9]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=30
n_3rdHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [10]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

8 0 0.0594844
8 1000 0.0934577
8 2000 0.0981473
6 3000 0.0996605
7 4000 0.0987898
7 5000 0.0974686
15 6000 0.0997668
2 7000 0.0975844
10 8000 0.103077
15 9000 0.10619
11 10000 0.104992
8 11000 0.103286
8 12000 0.103685
0 13000 0.111437
11 14000 0.102675
18 15000 0.106349
17 16000 0.102384
2 17000 0.120925
7 18000 0.111471
3 19000 0.118386
15 20000 0.100725
0 21000 0.11871
8 22000 0.116021
7 23000 0.122172
13 24000 0.126074
9 25000 0.114277
8 26000 0.111456
7 27000 0.120353
14 28000 0.10986
8 29000 0.115471
6 30000 0.11848
18 31000 0.119425
11 32000 0.117928
12 33000 0.112237
2 34000 0.113067
6 35000 0.113546
3 36000 0.111758
7 37000 0.109041
16 38000 0.115154
4 39000 0.122314
14 40000 0.118878
3 41000 0.113248
16 42000 0.115505
17 43000 0.11083
0 44000 0.110674
10 45000 0.11487
14 46000 0.115533
15 47000 0.11516
19 48000 0.111967
18 49000 0.115233
8 50000 0.116072
10 51000 0.114825
18 52000 0.112569
8 53000 0.11706
7 54000 0.116262
9 55000 0.116531
13 56000 0.118971
14 57000 0.113998
1

# 5th ReLU, tanh, 4, 50, 50, 50, 1
R2 = 0.587211552943

## Model

In [11]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=50
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [12]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

18 0 0.0593963
6 1000 0.0988289
18 2000 0.0957394
7 3000 0.0982848
12 4000 0.10646
18 5000 0.106831
12 6000 0.110742
14 7000 0.110526
0 8000 0.111224
15 9000 0.111403
13 10000 0.0876452
0 11000 0.113256
14 12000 0.115606
10 13000 0.11472
19 14000 0.114371
4 15000 0.109506
3 16000 0.111037
1 17000 0.115207
9 18000 0.116183
16 19000 0.11143
10 20000 0.11894
19 21000 0.116465
14 22000 0.116571
15 23000 0.114528
19 24000 0.113283
0 25000 0.114385
9 26000 0.113969
2 27000 0.114596
19 28000 0.114905
11 29000 0.114939
7 30000 0.115006
5 31000 0.114689
7 32000 0.114953
18 33000 0.115066
16 34000 0.114993
9 35000 0.11563
12 36000 0.113433
0 37000 0.121245
15 38000 0.114956
17 39000 0.114997
2 40000 0.113638
5 41000 0.116218
7 42000 0.118442
12 43000 0.119567
17 44000 0.115007
1 45000 0.115016
7 46000 0.115434
12 47000 0.116504
1 48000 0.114807
5 49000 0.115065
7 50000 0.115042
7 51000 0.114945
0 52000 0.115066
5 53000 0.115858
12 54000 0.115899
12 55000 0.114973
17 56000 0.115473
15 57000 0.114

# 6th eLU, tanh, 4, 50, 40, 30, 20, 1
R2 = 0.728673962973

## Model

In [17]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=40
n_3rdHiddenUnit=30
n_4thHiddenUnit=20
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [18]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

11 0 0.0584089
5 1000 0.0889852
3 2000 0.096943
19 3000 0.0955534
0 4000 0.0826085
14 5000 0.103788
4 6000 0.094176
1 7000 0.0958272
14 8000 0.105854
8 9000 0.110534
7 10000 0.100915
16 11000 0.118889
1 12000 0.09571
14 13000 0.112289
6 14000 0.121041
11 15000 0.110803
4 16000 0.1061
1 17000 0.108667
0 18000 0.104039
8 19000 0.112856
10 20000 0.147471
15 21000 0.105981
11 22000 0.124301
8 23000 0.114043
5 24000 0.115559
6 25000 0.11247
8 26000 0.105947
7 27000 0.111409
5 28000 0.118417
9 29000 0.122557
10 30000 0.107671
6 31000 0.113358
17 32000 0.121184
16 33000 0.109899
6 34000 0.105591
8 35000 0.114376
6 36000 0.115434
15 37000 0.115779
10 38000 0.118606
19 39000 0.126987
4 40000 0.111636
11 41000 0.115003
15 42000 0.0954767
0 43000 0.114065
6 44000 0.113682
6 45000 0.10701
0 46000 0.115171
1 47000 0.119999
1 48000 0.0927157
3 49000 0.113133
0 50000 0.110648
4 51000 0.120122
9 52000 0.11475
13 53000 0.112333
18 54000 0.124206
2 55000 0.114164
2 56000 0.11451
14 57000 0.114476
11 580

# 7th eLU, tanh, 4, 100, 100, 50, 1
R2 = 0.801217519573

## Model

In [6]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=100
n_2ndHiddenUnit=100
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [7]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.058191
1 1000 0.101107
12 2000 0.0960644
9 3000 0.0938841
7 4000 0.114192
11 5000 0.0969135
3 6000 0.0922015
9 7000 0.104336
10 8000 0.102777
12 9000 0.105459
6 10000 0.109769
5 11000 0.107603
17 12000 0.116835
6 13000 0.114466
15 14000 0.100625
5 15000 0.10978
15 16000 0.11109
8 17000 0.116498
6 18000 0.115498
1 19000 0.118965
16 20000 0.128822
12 21000 0.123297
0 22000 0.114232
11 23000 0.11745
19 24000 0.113926
19 25000 0.111348
6 26000 0.117462
3 27000 0.116889
18 28000 0.121879
10 29000 0.121835
12 30000 0.122406
2 31000 0.11689
17 32000 0.118096
12 33000 0.117157
18 34000 0.113255
18 35000 0.118077
10 36000 0.111037
7 37000 0.114608
6 38000 0.113373
17 39000 0.110088
17 40000 0.115716
1 41000 0.115978
13 42000 0.116177
13 43000 0.111218
7 44000 0.11466
1 45000 0.111398
4 46000 0.121792
5 47000 0.112257
10 48000 0.114788
13 49000 0.118285
10 50000 0.113396
15 51000 0.115431
19 52000 0.11741
9 53000 0.115719
17 54000 0.117874
3 55000 0.114671
13 56000 0.115848
6 57000 0.11569

# 8th eLU, tanh, 4, 10, 10, 10, 10, 10, 10, 1
R2 = 0.616077753531

## Model

In [8]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=10
n_3rdHiddenUnit=10
n_4thHiddenUnit=10
n_5thHiddenUnit=10
n_6thHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_5thHiddenUnit],stddev=0.01))
W6 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_6thHiddenUnit],stddev=0.01))
W7 = tf.Variable(tf.random_normal([n_6thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_5thHiddenUnit], stddev=0.01))
b6 = tf.Variable(tf.random_normal([n_6thHiddenUnit], stddev=0.01))
b7 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
L5 = tf.nn.elu(tf.matmul(L4, W5) + b5)
L6 = tf.nn.elu(tf.matmul(L5, W6) + b6)
Hypothesis = tf.nn.tanh(tf.matmul(L6, W7) + b7)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [9]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

7 0 0.0584986
4 1000 0.05751
0 2000 0.0575096
9 3000 0.0575098
2 4000 0.0575105
19 5000 0.0575096
1 6000 0.0575096
6 7000 0.0575095
10 8000 0.0910533
6 9000 0.0927686
14 10000 0.094628
19 11000 0.0938583
8 12000 0.0980775
19 13000 0.0966723
2 14000 0.0971877
7 15000 0.0984727
9 16000 0.104427
19 17000 0.0991159
12 18000 0.10123
19 19000 0.106698
6 20000 0.101266
2 21000 0.105709
18 22000 0.108244
1 23000 0.103687
18 24000 0.10294
11 25000 0.106746
0 26000 0.101528
6 27000 0.109472
16 28000 0.111191
16 29000 0.108533
6 30000 0.115233
10 31000 0.1034
14 32000 0.114603
11 33000 0.103743
7 34000 0.109129
19 35000 0.119815
6 36000 0.110349
7 37000 0.10667
9 38000 0.113066
12 39000 0.110166
2 40000 0.105086
9 41000 0.1098
15 42000 0.110773
0 43000 0.10842
9 44000 0.115677
7 45000 0.116446
7 46000 0.111838
5 47000 0.105516
15 48000 0.108791
17 49000 0.111241
12 50000 0.10988
10 51000 0.109114
12 52000 0.113994
14 53000 0.10778
7 54000 0.115579
0 55000 0.114109
10 56000 0.108697
18 57000 0.112

# 9th eLU, tanh, 4, 500, 500, 100, 1
R2 = 0.76630321537

## Model

In [10]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=500
n_2ndHiddenUnit=500
n_3rdHiddenUnit=100
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [11]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

9 0 0.0591716
9 1000 0.0892799
16 2000 0.10809
3 3000 0.103186
7 4000 0.103598
14 5000 0.117744
16 6000 0.127433
1 7000 0.114091
19 8000 0.104694
1 9000 0.116884
14 10000 0.141292
8 11000 0.120442
10 12000 0.127927
2 13000 0.116145
18 14000 0.130627
8 15000 0.110261
12 16000 0.111499
1 17000 0.112028
13 18000 0.116446
10 19000 0.108579
0 20000 0.112549
9 21000 0.121002
7 22000 0.117856
11 23000 0.11388
1 24000 0.114377
14 25000 0.11183
6 26000 0.128711
10 27000 0.124649
0 28000 0.112747
2 29000 0.126037
5 30000 0.116953
8 31000 0.10597
14 32000 0.119594
16 33000 0.108452
10 34000 0.100466
13 35000 0.108436
9 36000 0.122141
2 37000 0.115701
7 38000 0.11209
6 39000 0.0956433
10 40000 0.1147
13 41000 0.114312
15 42000 0.118883
12 43000 0.115445
2 44000 0.116235
14 45000 0.112408
3 46000 0.113826
18 47000 0.115152
5 48000 0.113972
10 49000 0.18944
7 50000 0.114084
19 51000 0.10623
2 52000 0.116874
12 53000 0.108805
16 54000 0.122585
7 55000 0.116312
4 56000 0.107072
4 57000 0.116556
1 5800

# 10th eLU, tanh, 4, 100, 100, 100, 50, 1
R2 = 0.834434007079

## Model

In [4]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=100
n_2ndHiddenUnit=100
n_3rdHiddenUnit=100
n_4thHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [5]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0591532
9 1000 0.0819874
3 2000 0.111594
18 3000 0.0960085
0 4000 0.101961
15 5000 0.0962025
15 6000 0.102826
18 7000 0.109316
4 8000 0.101001
5 9000 0.1166
9 10000 0.0995242
3 11000 0.11739
15 12000 0.116031
11 13000 0.119342
15 14000 0.102046
9 15000 0.128017
14 16000 0.114145
10 17000 0.10797
11 18000 0.118899
10 19000 0.112987
5 20000 0.113118
2 21000 0.118085
9 22000 0.119887
4 23000 0.117802
15 24000 0.111444
0 25000 0.110789
1 26000 0.114857
0 27000 0.114896
7 28000 0.118763
3 29000 0.112085
19 30000 0.107129
3 31000 0.114253
1 32000 0.114603
12 33000 0.114935
6 34000 0.124381
1 35000 0.113026
11 36000 0.114894
0 37000 0.115068
14 38000 0.115593
0 39000 0.115816
11 40000 0.108769
18 41000 0.114977
3 42000 0.116739
17 43000 0.11549
4 44000 0.112316
4 45000 0.11509
19 46000 0.148605
17 47000 0.115076
9 48000 0.115063
15 49000 0.113165
15 50000 0.11599
3 51000 0.101449
15 52000 0.115462
3 53000 0.112869
19 54000 0.115575
14 55000 0.11646
17 56000 0.115033
2 57000 0.114915
6 5